# OpenAI Fine-tuning API "Neo Optimizer"

### import modules

In [10]:
from openai import OpenAI
from dotenv import load_dotenv
import csv
import json
import random
import os
load_dotenv()

True

### create client

In [11]:
my_skey = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=my_skey)

### prepare training data

In [30]:
# load csv of YouTube comments

query_list = []
optimized_query_list = []

with open('dataDataSet.csv', mode ='r') as file:
    file = csv.reader(file)
    
    # read file line by line
    for line in file:
        # skip first line
        if line[0]=='query':
            continue
            
        # append comments and responses to respective lists
        query_list.append(line[0])
        optimized_query_list.append(line[1] + " -ShawGPT")

In [31]:
len(query_list)

38

In [38]:
# construct training examples
example_list = []

intstructions_string_few_shot = """ShawGPT, You are a chatbot specializing in optimizing SQL queries within the Oracle syntax ecosystem. Your primary functionality is to receive SQL queries from users and provide them with optimized versions for better performance. \

Here are examples of ShawGPT responding to queries.

query: SELECT * FROM Professor WHERE first_name = 'John';
ShawGPT: SELECT professor_id, last_name FROM Professor WHERE first_name = 'John' INDEX(first_name);  -ShawGPT

query: SELECT * FROM Course WHERE course_code = 'MATH101';
ShawGPT: SELECT course_id, course_name, credit_hours FROM Course WHERE course_code = 'MATH101' INDEX(course_code);  -ShawGPT

query: SELECT * FROM Enrollment WHERE student_id IN (10, 20, 30);
ShawGPT: SELECT course_id FROM Enrollment WHERE student_id IN (10, 20, 30) INDEX(student_id); -ShawGPT"""

for i in range(len(query_list)):    
    system_dict = {"role": "system", "content": intstructions_string_few_shot}
    user_dict = {"role": "user", "content": query_list[i]}
    assistant_dict = {"role": "assistant", "content": optimized_query_list[i]}
    
    messages_list = [system_dict, user_dict, assistant_dict]
    
    example_list.append({"messages": messages_list})

In [39]:
# create train/validation split
validation_index_list = random.sample(range(0, len(example_list)-1), 9)

validation_data_list = [example_list[index] for index in validation_index_list]

for example in validation_data_list:
    example_list.remove(example)

In [40]:
# write examples to file
with open('data/training-data.jsonl', 'w') as training_file:
    for example in example_list:
        json.dump(example, training_file)
        training_file.write('\n')

with open('data/validation-data.jsonl', 'w') as validation_file:
    for example in validation_data_list:
        json.dump(example, validation_file)
        validation_file.write('\n')

### upload training examples to openai api

In [41]:
training_file = client.files.create(
  file = open("data/training-data.jsonl", "rb"),
  purpose = "fine-tune"
)

validation_file = client.files.create(
  file = open("data/validation-data.jsonl", "rb"),
  purpose = "fine-tune"
)

### create a fine-tuned model

In [42]:
client.fine_tuning.jobs.create(
    training_file = training_file.id,
    validation_file = validation_file.id,
    suffix = "ShawGPT",
    model = "gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-zbSaFTFwBvPjjZPTh4Sj4Zgo', created_at=1707660200, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0613', object='fine_tuning.job', organization_id='org-K9BB640pBH3H5vq7s9XXo0zd', result_files=[], status='validating_files', trained_tokens=None, training_file='file-tyf1U4QNbQqH34aADKIBTUPu', validation_file='file-Y4Sn2L90xwlmL1miMFjUWxNO')

### use fine-tuned model

In [63]:
test_query = """SELECT employees.employee_id, employees.first_name, employees.last_name, departments.department_name
FROM employees
JOIN departments ON employees.department_id = departments.department_id
WHERE employees.salary > 50000;"""

In [64]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
    {"role": "system", "content": intstructions_string_few_shot},
    {"role": "user", "content": test_query}
    ]
)

In [65]:
print(dict(response)['choices'][0].message.content)

The query you provided looks fine. However, it could benefit from an index on the "salary" column of the "employees" table to improve performance. Here's the optimized version:

SELECT employees.employee_id, employees.first_name, employees.last_name, departments.department_name
FROM employees INDEX(salary)
JOIN departments ON employees.department_id = departments.department_id
WHERE employees.salary > 50000;

Adding INDEX(salary) ensures that the database engine uses an index scan on the "salary" column, which can speed up the query execution.


In [1]:
# delete file
# client.files.delete(training_file.id)
# client.files.delete(validation_file.id)

### More resources

OpenAI Guide: https://platform.openai.com/docs/guides/fine-tuning <br>
Fine-tuning doc: https://platform.openai.com/docs/api-reference/fine-tuning <br>
Fine-tuning data prep: https://cookbook.openai.com/examples/chat_finetuning_data_prep